# Накопленный охват в динамике по ФО
Пример расчета показателей телеканалов в разбивке по федеральным округам и месяцам.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 30.09.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разбивке по федеральным округам
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ									
- Статистики: Reach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


Обратившись к каталогу демографических переменных, получим entityName переменной Федеральные округа

In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['федерал'])

Запрошены записи: 0 - 8
Всего найдено записей: 8



,id,valueId,name,colName,valueName
0,45,5,Федеральные округа,FederalOkrug,УРАЛЬСКИЙ
1,45,6,Федеральные округа,FederalOkrug,СИБИРСКИЙ
2,45,2,Федеральные округа,FederalOkrug,СЕВЕРО-ЗАПАДНЫЙ
3,45,7,Федеральные округа,FederalOkrug,ДАЛЬНЕВОСТОЧНЫЙ
4,45,3,Федеральные округа,FederalOkrug,ЮЖНЫЙ
5,45,8,Федеральные округа,FederalOkrug,СЕВЕРО-КАВКАЗСКИЙ
6,45,1,Федеральные округа,FederalOkrug,ЦЕНТРАЛЬНЫЙ
7,45,4,Федеральные округа,FederalOkrug,ПРИВОЛЖСКИЙ


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-01-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId = 1'

# Указываем список статистик для расчета
statistics = ['ReachPer']

# Указываем срезы
slices = ['federalOkrug', 'researchMonth']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

## Расчет задания

In [5]:
# Формируем задание для TV API в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))
df

Расчет задачи (id: 679db5db-ce3a-43f7-8b36-727b5a9fa6bf) [= = = = = ] время расчета: 0:00:16.681898


,researchMonth,federalOkrug,ReachPer
0,2022-01-01,СЕВЕРО-ЗАПАДНЫЙ,82.463601
1,2022-01-01,ЮЖНЫЙ,79.519220
2,2022-01-01,ПРИВОЛЖСКИЙ,80.300861
3,2022-01-01,УРАЛЬСКИЙ,79.681112
4,2022-01-01,СИБИРСКИЙ,79.203791
...,...,...,...
67,2022-09-01,УРАЛЬСКИЙ,75.030053
68,2022-09-01,СИБИРСКИЙ,69.759466
69,2022-09-01,ДАЛЬНЕВОСТОЧНЫЙ,72.875819
70,2022-09-01,СЕВЕРО-КАВКАЗСКИЙ,71.150772


Настраиваем внешний вид таблицы

In [6]:
df_res = pd.pivot_table(df, values=['ReachPer'],
                        index=['federalOkrug'], 
                        columns=['researchMonth'])
df_res

ReachPer                                              \
researchMonth     2022-01-01 2022-02-01 2022-03-01 2022-04-01 2022-05-01   
federalOkrug                                                               
ДАЛЬНЕВОСТОЧНЫЙ    77.206884  76.724191  78.756855  73.991365  77.423981   
ПРИВОЛЖСКИЙ        80.300861  79.855787  78.636915  74.151029  75.968877   
СЕВЕРО-ЗАПАДНЫЙ    82.463601  79.741623  78.783721  72.990341  76.552241   
СЕВЕРО-КАВКАЗСКИЙ  78.702214  75.230596  81.809122  73.955807  71.713377   
СИБИРСКИЙ          79.203791  78.410506  75.788869  70.089647  73.294431   
УРАЛЬСКИЙ          79.681112  76.858929  77.649050  71.171893  74.377031   
ЦЕНТРАЛЬНЫЙ        77.546158  77.860537  77.035497  74.624606  76.293177   
ЮЖНЫЙ              79.519220  78.834196  80.158703  75.964578  77.072823   

                                                               
researchMonth     2022-06-01 2022-07-01 2022-08-01 2022-09-01  
federalOkrug                                                   
ДАЛЬНЕВОСТОЧНЫЙ    73.595095  73.872464  73.552728  72.875819  
ПРИВОЛЖСКИЙ        72.068095  70.245781  70.322699  71.878380  
СЕВЕРО-ЗАПАДНЫЙ    70.278012  72.125218  67.759949  70.589911  
СЕВЕРО-КАВКАЗСКИЙ  69.716659  66.419149  62.635762  71.150772  
СИБИРСКИЙ          66.957993  68.983723  67.487760  69.759466  
УРАЛЬСКИЙ          69.814429  71.212038  66.793931  75.030053  
ЦЕНТРАЛЬНЫЙ        70.605133  71.245580  69.808742  74.026687  
ЮЖНЫЙ              72.348608  71.634835  68.462899  76.031039

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [7]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_reach_by_federal_okrug'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()